In [1]:
import pandas as pd
import json
import os
import sys
import tqdm
import time
import datasets

In [3]:
data_path="/home/aberhe/Projects/SANTAL/ASLR/data/ner"

In [4]:
os.listdir(data_path)

['bc5cdr',
 'BC5CDR-chem',
 'BC5CDR-disease',
 'JNLPBA',
 'linnaeus',
 'NCBI-disease',
 's800']

In [5]:
os.listdir(os.path.join(data_path,"bc5cdr"))

['dev.txt', 'test.txt', 'train.txt']

In [120]:
with open(os.path.join(os.path.join(data_path,"BC5CDR-chem"),"train.txt"),"r") as f:
    lines=f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: '/home/aberhe/Projects/SANTAL/ASLR/data/ner/BC5CDR-chem/train.txt'

In [119]:
lines[2:20]

['IL-2\tNN\tO\tB-DNA\n',
 'gene\tNN\tO\tI-DNA\n',
 'expression\tNN\tO\tO\n',
 'and\tNN\tO\tO\n',
 'NF-kappa\tNN\tO\tB-protein\n',
 'B\tNN\tO\tI-protein\n',
 'activation\tNN\tO\tO\n',
 'through\tNN\tO\tO\n',
 'CD28\tNN\tO\tB-protein\n',
 'requires\tNN\tO\tO\n',
 'reactive\tNN\tO\tO\n',
 'oxygen\tNN\tO\tO\n',
 'production\tNN\tO\tO\n',
 'by\tNN\tO\tO\n',
 '5-lipoxygenase\tNN\tO\tB-protein\n',
 '.\tNN\tO\tO\n',
 '\n',
 'Activation\tNN\tO\tO\n']

In [ ]:
###  bc5cdr,JNLPBA= "-DOCSTART-""
### BC5CDR-disease, BC5CDR-chem,s800="\n"
### linnaeus, NCBI-disease="Abstarct"

In [126]:
def getDocSequences(path_to_file,entity_label="Chemical"):
    with open(path_to_file,"r") as f:
        lines=f.readlines()
    doc_seq=[]
    doc_labels=[]
    word_seq=[]
    label_seq=[]
    for i in range (1,len(lines)):
        if lines[i]!="\n":
            if not "-DOCSTART-" in lines[i]:
                word_seq.append(lines[i].split("\t")[0])
                if "-" in lines[i].split("\t")[-1]:
                    label_seq.append(lines[i].split("\t")[-1].replace("\n",""))#.split("-")[0]+"-"+entity_label)
                else:
                    label_seq.append(lines[i].split("\t")[-1].replace("\n",""))             
            else:
                if word_seq!=[]:
                    doc_seq.append(word_seq)
                    doc_labels.append(label_seq)
                label_seq=[]
                word_seq=[]
    
    return doc_seq,doc_labels

In [9]:
def cleanBrackets(doc_seq,doc_labels):
    doc_seq_cl,doc_labels_cl=[],[]
    pending_bracket=None
    previous=None
    c=0
    for i in range(1,len(doc_seq)):#,doc_labels):
        if "[" in doc_seq[i] or "(" in doc_seq[i] or "{" in doc_seq[i]:
            if "]" in doc_seq[i] or ")" in doc_seq[i] or "}" in doc_seq[i]:
                joined_item=previous+" ".join(doc_seq[i-c:i+1])
                doc_seq_cl.append(joined_item)
                doc_labels_cl.append(doc_labels[i])
                c=0
                doc_seq_cl.append(joined_item)
                #previous=doc_seq[i]
            else:
                previous=doc_seq[i]
                c=c+1
        else:
            doc_seq_cl.append(doc_seq[i])
            doc_labels_cl.append(doc_labels[i])
            c=0

    return doc_seq_cl,doc_labels_cl



In [147]:
file_path=os.path.join(os.path.join(data_path,"s800"),"train.tsv")
doc_seq,doc_labels=getDocSequences(path_to_file=file_path,entity_label="Species")
file_path=os.path.join(os.path.join(data_path,"s800"),"test.tsv")
doc_seq_t,doc_labels_t=getDocSequences(path_to_file=file_path,entity_label="Species")
file_path=os.path.join(os.path.join(data_path,"s800"),"devel.tsv")
doc_seq_v,doc_labels_v=getDocSequences(path_to_file=file_path,entity_label="Species")

In [148]:
len(doc_seq),len(doc_labels),len(doc_seq_t),len(doc_labels_t),len(doc_seq_v),len(doc_labels_v)

(0, 0, 0, 0, 0, 0)

In [130]:

doc_seq[0]

['IL-2',
 'gene',
 'expression',
 'and',
 'NF-kappa',
 'B',
 'activation',
 'through',
 'CD28',
 'requires',
 'reactive',
 'oxygen',
 'production',
 'by',
 '5-lipoxygenase',
 '.',
 'Activation',
 'of',
 'the',
 'CD28',
 'surface',
 'receptor',
 'provides',
 'a',
 'major',
 'costimulatory',
 'signal',
 'for',
 'T',
 'cell',
 'activation',
 'resulting',
 'in',
 'enhanced',
 'production',
 'of',
 'interleukin-2',
 '(',
 'IL-2',
 ')',
 'and',
 'cell',
 'proliferation',
 '.',
 'In',
 'primary',
 'T',
 'lymphocytes',
 'we',
 'show',
 'that',
 'CD28',
 'ligation',
 'leads',
 'to',
 'the',
 'rapid',
 'intracellular',
 'formation',
 'of',
 'reactive',
 'oxygen',
 'intermediates',
 '(',
 'ROIs',
 ')',
 'which',
 'are',
 'required',
 'for',
 'CD28',
 '-mediated',
 'activation',
 'of',
 'the',
 'NF-kappa',
 'B',
 '/',
 'CD28-responsive',
 'complex',
 'and',
 'IL-2',
 'expression',
 '.',
 'Delineation',
 'of',
 'the',
 'CD28',
 'signaling',
 'cascade',
 'was',
 'found',
 'to',
 'involve',
 'protein

In [131]:
doc_labels[1]

['O',
 'B-DNA',
 'I-DNA',
 'I-DNA',
 'O',
 'B-DNA',
 'I-DNA',
 'I-DNA',
 'I-DNA',
 'I-DNA',
 'I-DNA',
 'O',
 'O',
 'B-cell_type',
 'O',
 'O',
 'O',
 'B-cell_type',
 'I-cell_type',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DNA',
 'I-DNA',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DNA',
 'I-DNA',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-protein',
 'I-protein',
 'B-protein',
 'I-protein',
 'O',
 'O',
 'O',
 'B-DNA',
 'I-DNA',
 'I-DNA',
 'O',
 'O',
 'B-DNA',
 'I-DNA',
 'I-DNA',
 'I-DNA',
 'O',
 'O',
 'O',
 'O',
 'B-DNA',
 'I-DNA',
 'O',
 'B-cell_type',
 'O',
 'B-cell_type',
 'I-cell_type',
 'O',
 'O',
 'O',
 'O',
 'B-DNA',
 'I-DN

In [87]:
doc_seq_v[0]

['Tricuspid',
 'valve',
 'regurgitation',
 'and',
 'lithium',
 'carbonate',
 'toxicity',
 'in',
 'a',
 'newborn',
 'infant',
 '.',
 'A',
 'newborn',
 'with',
 'massive',
 'tricuspid',
 'regurgitation',
 ',',
 'atrial',
 'flutter',
 ',',
 'congestive',
 'heart',
 'failure',
 ',',
 'and',
 'a',
 'high',
 'serum',
 'lithium',
 'level',
 'is',
 'described',
 '.',
 'This',
 'is',
 'the',
 'first',
 'patient',
 'to',
 'initially',
 'manifest',
 'tricuspid',
 'regurgitation',
 'and',
 'atrial',
 'flutter',
 ',',
 'and',
 'the',
 '11th',
 'described',
 'patient',
 'with',
 'cardiac',
 'disease',
 'among',
 'infants',
 'exposed',
 'to',
 'lithium',
 'compounds',
 'in',
 'the',
 'first',
 'trimester',
 'of',
 'pregnancy',
 '.',
 'Lithium',
 'carbonate',
 'may',
 'be',
 'a',
 'factor',
 'in',
 'the',
 'increasing',
 'incidence',
 'of',
 'congenital',
 'heart',
 'disease',
 'when',
 'taken',
 'during',
 'early',
 'pregnancy',
 '.',
 'It',
 'also',
 'causes',
 'neurologic',
 'depression',
 ',',
 'c

In [168]:
len_docs=[len(doc) for doc in doc_seq]
print(len_docs)
max(len_docs),min(len_docs)

[201, 67, 131, 86, 291, 204, 61, 75, 182, 149, 92, 149, 92, 137, 252, 133, 243, 97, 212, 157, 120, 219, 221, 68, 298, 108, 81, 108, 117, 51, 153, 206, 194, 130, 142, 209, 123, 262, 59, 168, 101, 125, 152, 152, 110, 291, 217, 117, 63, 116, 96, 104, 215, 346, 610, 352, 49, 125, 169, 392, 318, 256, 265, 325, 128, 331, 51, 286, 193, 167, 323, 326, 293, 166, 315, 166, 126, 257, 235, 105, 311, 278, 379, 210, 211, 133, 97, 260, 249, 213, 81, 197, 72, 158, 177, 104, 72, 157, 84, 176, 119, 216, 72, 182, 55, 57, 138, 77, 88, 109, 74, 242, 225, 287, 183, 99, 128, 371, 102, 266, 274, 267, 167, 238, 191, 144, 147, 267, 242, 35, 57, 233, 137, 141, 155, 277, 279, 231, 128, 355, 51, 357, 305, 271, 130, 327, 63, 93, 321, 281, 179, 252, 147, 164, 196, 115, 312, 154, 210, 293, 173, 311, 200, 283, 154, 99, 250, 246, 168, 94, 251, 143, 354, 203, 242, 137, 225, 164, 288, 277, 121, 153, 249, 202, 168, 122, 94, 146, 159, 147, 182, 62, 363, 136, 194, 84, 67, 97, 86, 101, 226, 108, 184, 100, 187, 236, 156, 94, 

(610, 26)

In [163]:
ct=0
for l in len_docs:
    if l>300:
        ct=ct+1
print(ct,len(doc_seq))

74 499


In [11]:
def isclosed(text):
    """
    Check is there is an opening bracket but not a closing bracket
    """
    openings=['(','{','[']
    closings=[')','}',']']
    close=1
    for i,op in enumerate(openings):
        if op in text:
            if closings[i] in text:
                close=1
            else:
                close=0
                return False
        else:
            continue
    if close:
        return True
    else:
        return False
    
    print("no brackets")



In [12]:
isclosed("[")

False

In [222]:
def getCleanEntities(doc_seq,doc_labels):
    entities_list=[]
    labels_list=[]
    for d in tqdm.tqdm(range(len(doc_seq))):
        entities=[]
        labels=[]
        text=doc_seq[d][0]
        lbl=doc_labels[d][0]
        for  i in range(1,len(doc_seq[d])):
            #if "Chemical" in doc_labels[0][i]:
            #print(i,":",doc_seq[d][i],":",doc_labels[d][i])
            if isclosed(text):
                entities.append(text)
                labels.append(lbl)
                text=doc_seq[d][i]
                lbl=doc_labels[d][i]
            else:
                text=text+doc_seq[d][i]
                lbl=doc_labels[d][i]
        entities.append(text)
        labels.append(lbl)
        entities_list.append(entities)
        labels_list.append(labels)

    return entities_list,labels_list

In [82]:
entities_list,labels_list=getCleanEntities(doc_seq=doc_seq,doc_labels=doc_labels)
entities_list_test,labels_list_test=getCleanEntities(doc_seq=doc_seq_t,doc_labels=doc_labels_t)
entities_list_valid,labels_list_valid=getCleanEntities(doc_seq=doc_seq_v,doc_labels=doc_labels_v)

100%|██████████| 499/499 [00:00<00:00, 9672.87it/s]


In [88]:
dic_train=datasets.Dataset.from_dict({"tokens":entities_list,"ner_tags":labels_list})
dic_test=datasets.Dataset.from_dict({"tokens":entities_list_test,"ner_tags":labels_list_test})
dic_val=datasets.Dataset.from_dict({"tokens":entities_list_valid,"ner_tags":labels_list_valid})
chemical_dataset=datasets.DatasetDict({"train":dic_train,"valid":dic_val,"test":dic_test})

In [96]:
chemical_dataset=datasets.DatasetDict({"train":dic_train,"valid":dic_val,"test":dic_test})

In [106]:
chemical_dataset.save_to_disk(os.path.join(data_path,"Chemical_bc5cdr_dataset_ner"))

Saving the dataset (0/1 shards):   0%|          | 0/499 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/499 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/499 [00:00<?, ? examples/s]

In [90]:
chemical_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1497
})

In [61]:
d=0
for i in range(len(doc_seq[d])):
    print(doc_seq[d][i],"--",doc_labels[d][i],end=" -->")


Naloxone -- I-Chemical -->reverses -- O -->the -- O -->antihypertensive -- O -->effect -- O -->of -- O -->clonidine -- I-Chemical -->. -- O -->In -- O -->unanesthetized -- O -->, -- O -->spontaneously -- O -->hypertensive -- I-Chemical -->rats -- O -->the -- O -->decrease -- O -->in -- O -->blood -- O -->pressure -- O -->and -- O -->heart -- O -->rate -- O -->produced -- O -->by -- O -->intravenous -- O -->clonidine -- I-Chemical -->, -- O -->5 -- O -->to -- O -->20 -- O -->micrograms -- O -->/ -- O -->kg -- O -->, -- O -->was -- O -->inhibited -- O -->or -- O -->reversed -- O -->by -- O -->nalozone -- I-Chemical -->, -- O -->0.2 -- O -->to -- O -->2 -- O -->mg -- O -->/ -- O -->kg -- O -->. -- O -->The -- O -->hypotensive -- I-Chemical -->effect -- O -->of -- O -->100 -- O -->mg -- O -->/ -- O -->kg -- O -->alpha -- B-Chemical -->- -- I-Chemical -->methyldopa -- I-Chemical -->was -- O -->also -- O -->partially -- O -->reversed -- O -->by -- O -->naloxone -- I-Chemical -->. -- O -->Nal

In [79]:
#print(entities_list1[0])
#print(labels_list1[0])
for i in range(len(entities_list[0])):
    print(entities_list[0][i],"--",labels_list[0][i],end=" -->")

Naloxone -- I-Chemical -->reverses -- O -->the -- O -->antihypertensive -- O -->effect -- O -->of -- O -->clonidine -- I-Chemical -->. -- O -->In -- O -->unanesthetized -- O -->, -- O -->spontaneously -- O -->hypertensive -- I-Chemical -->rats -- O -->the -- O -->decrease -- O -->in -- O -->blood -- O -->pressure -- O -->and -- O -->heart -- O -->rate -- O -->produced -- O -->by -- O -->intravenous -- O -->clonidine -- I-Chemical -->, -- O -->5 -- O -->to -- O -->20 -- O -->micrograms -- O -->/ -- O -->kg -- O -->, -- O -->was -- O -->inhibited -- O -->or -- O -->reversed -- O -->by -- O -->nalozone -- I-Chemical -->, -- O -->0.2 -- O -->to -- O -->2 -- O -->mg -- O -->/ -- O -->kg -- O -->. -- O -->The -- O -->hypotensive -- I-Chemical -->effect -- O -->of -- O -->100 -- O -->mg -- O -->/ -- O -->kg -- O -->alpha -- B-Chemical -->- -- I-Chemical -->methyldopa -- I-Chemical -->was -- O -->also -- O -->partially -- O -->reversed -- O -->by -- O -->naloxone -- I-Chemical -->. -- O -->Nal

In [53]:
print(doc_labels[0])

['I-Chemical', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'I-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'B-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'I-Chemical', 'O', 'O', '

In [81]:
for i,(e,l) in enumerate(zip(entities_list[0],labels_list[0])):
    if "Chemical" in l:
        print(i,e,l)

0 Naloxone I-Chemical
6 clonidine I-Chemical
12 hypertensive I-Chemical
25 clonidine I-Chemical
39 nalozone I-Chemical
49 hypotensive I-Chemical
56 alpha B-Chemical
57 - I-Chemical
58 methyldopa I-Chemical
64 naloxone I-Chemical
66 Naloxone I-Chemical
83 hypertensive I-Chemical
85 clonidine I-Chemical
98 [3H]-naloxone I-Chemical
102 naloxone I-Chemical
112 clonidine I-Chemical
117 [3H]-dihydroergocryptine I-Chemical
126 hypertensive I-Chemical
143 naloxone I-Chemical
145 clonidine I-Chemical
169 clonidine I-Chemical
171 alpha B-Chemical
172 - I-Chemical
173 methyldopa I-Chemical


In [172]:
print(doc_labels[0][doc_seq[0].index('3H]-naloxone')-1])

B-Chemical


In [80]:
len(doc_labels)

499

In [60]:
sentences=[]
sent=""

for i in range (2,len(lines)):
    if not lines[i].split("\t")[0] in ["?",".","!"] and "-DOCSTART-" in lines[i]:
        sent=sent+" "+lines[i].split("\t")[0]
    else:
        sent=sent+""+lines[i].split("\t")[0]
        sentences.append(sent.strip().replace(" , ",", ").replace(" / ","/"))
        sent=""

In [61]:
sentences[0:10]

['Naloxone reverses the antihypertensive effect of clonidine.',
 'In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg.',
 'The hypotensive effect of 100 mg/kg alpha - methyldopa was also partially reversed by naloxone.',
 'Naloxone alone did not affect either blood pressure or heart rate.',
 'In brain membranes from spontaneously hypertensive rats clonidine, 10(-8 ) to 10(-5 ) \n M, did not influence stereoselective binding of [ 3H]-naloxone ( 8 nM ), and naloxone, 10(-8 ) to 10(-4 ) \n M, did not influence clonidine - suppressible binding of [ 3H]-dihydroergocryptine ( 1 nM ).',
 'These findings indicate that in spontaneously hypertensive rats the effects of central alpha - adrenoceptor stimulation involve activation of opiate receptors.',
 'As naloxone and clonidine do not appear to interact with the same receptor site, the obser

In [206]:
def readannotation_tsv(filepath):
    with open (filepath, "r") as f:
        lines=f.readlines()
    c=0
    doc_seq=[]
    doc_labels=[]
    seq=[]
    lbl=[]
    print(lines[0])
    for line in lines:
        if line !="\n":
            split_line=line.split("\t")
            seq.append(split_line[0])
            lbl.append(split_line[-1])
        else:
            doc_seq.append(seq)
            doc_labels.append(lbl)
            seq=[]
            lbl=[]
    return doc_seq,doc_labels

In [229]:
doc_seq,doc_labels=readannotation_tsv(os.path.join(os.path.join(data_path,"s800"),"train.tsv"))
doc_seq_t,doc_labels_t=readannotation_tsv(os.path.join(os.path.join(data_path,"s800"),"test.tsv"))
doc_seq_v,doc_labels_v=readannotation_tsv(os.path.join(os.path.join(data_path,"s800"),"devel.tsv"))

Methanoregula	B

Streptococcus	B

Flavobacterium	O



In [230]:
len(doc_seq),len(doc_labels),len(doc_seq_v),len(doc_labels_v),len(doc_seq_t),len(doc_labels_t)

(5733, 5733, 830, 830, 1630, 1630)

In [232]:
doc_seq[0]

['Methanoregula',
 'formicica',
 'sp',
 '.',
 'nov',
 '.',
 ',',
 'a',
 'methane',
 '-',
 'producing',
 'archaeon',
 'isolated',
 'from',
 'methanogenic',
 'sludge',
 '.']

In [233]:
entities_list,labels_list=getCleanEntities(doc_seq=doc_seq,doc_labels=doc_labels)
entities_list_test,labels_list_test=getCleanEntities(doc_seq=doc_seq_t,doc_labels=doc_labels_t)
entities_list_valid,labels_list_valid=getCleanEntities(doc_seq=doc_seq_v,doc_labels=doc_labels_v)

100%|██████████| 830/830 [00:00<00:00, 86474.05it/s]


In [234]:
entities_list[0]

['Methanoregula',
 'formicica',
 'sp',
 '.',
 'nov',
 '.',
 ',',
 'a',
 'methane',
 '-',
 'producing',
 'archaeon',
 'isolated',
 'from',
 'methanogenic',
 'sludge',
 '.']

In [235]:
dic_train=datasets.Dataset.from_dict({"tokens":entities_list,"ner_tags":labels_list})
dic_test=datasets.Dataset.from_dict({"tokens":entities_list_test,"ner_tags":labels_list_test})
dic_val=datasets.Dataset.from_dict({"tokens":entities_list_valid,"ner_tags":labels_list_valid})
species_dataset=datasets.DatasetDict({"train":dic_train,"valid":dic_val,"test":dic_test})

In [236]:
species_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5733
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 830
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1630
    })
})

In [237]:
species_dataset.save_to_disk(os.path.join(data_path,"Species_s800_dataset_ner"))

Saving the dataset (0/1 shards):   0%|          | 0/5733 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/830 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1630 [00:00<?, ? examples/s]

In [242]:
doc_seq,doc_labels=readannotation_tsv(os.path.join(os.path.join(data_path,"linnaeus"),"train.tsv"))
doc_seq_t,doc_labels_t=readannotation_tsv(os.path.join(os.path.join(data_path,"linnaeus"),"test.tsv"))
doc_seq_v,doc_labels_v=readannotation_tsv(os.path.join(os.path.join(data_path,"linnaeus"),"devel.tsv"))
#doc_seq_d,doc_labels_d=readannotation_tsv(os.path.join(os.path.join(data_path,"linnaeus"),"train_dev.tsv"))

Scp160p	O

In	O

Gene	O



In [243]:
len(doc_seq),len(doc_labels),len(doc_seq_v),len(doc_labels_v),len(doc_seq_t),len(doc_labels_t)

(11935, 11935, 4078, 4078, 7142, 7142)

In [247]:
doc_seq[2]

['Scp160p',
 'is',
 'a',
 '160',
 'kDa',
 'protein',
 'in',
 'the',
 'yeast',
 'Saccharomyces',
 'cerevisiae',
 'that',
 'contains',
 '14',
 'repeats',
 'of',
 'the',
 'hnRNP',
 'K',
 '-',
 'homology',
 '(',
 'KH',
 ')',
 'domain',
 ',',
 'and',
 'demonstrates',
 'significant',
 'sequence',
 'homology',
 'to',
 'a',
 'family',
 'of',
 'proteins',
 'collectively',
 'known',
 'as',
 'vigilins',
 '.']

In [244]:
entities_list,labels_list=getCleanEntities(doc_seq=doc_seq,doc_labels=doc_labels)
entities_list_test,labels_list_test=getCleanEntities(doc_seq=doc_seq_t,doc_labels=doc_labels_t)
entities_list_valid,labels_list_valid=getCleanEntities(doc_seq=doc_seq_v,doc_labels=doc_labels_v)

100%|██████████| 4078/4078 [00:00<00:00, 116266.10it/s]


In [248]:
entities_list[2]

['Scp160p',
 'is',
 'a',
 '160',
 'kDa',
 'protein',
 'in',
 'the',
 'yeast',
 'Saccharomyces',
 'cerevisiae',
 'that',
 'contains',
 '14',
 'repeats',
 'of',
 'the',
 'hnRNP',
 'K',
 '-',
 'homology',
 '(KH)',
 'domain',
 ',',
 'and',
 'demonstrates',
 'significant',
 'sequence',
 'homology',
 'to',
 'a',
 'family',
 'of',
 'proteins',
 'collectively',
 'known',
 'as',
 'vigilins',
 '.']

In [249]:
dic_train=datasets.Dataset.from_dict({"tokens":entities_list,"ner_tags":labels_list})
dic_test=datasets.Dataset.from_dict({"tokens":entities_list_test,"ner_tags":labels_list_test})
dic_val=datasets.Dataset.from_dict({"tokens":entities_list_valid,"ner_tags":labels_list_valid})
species_dataset_lin=datasets.DatasetDict({"train":dic_train,"valid":dic_val,"test":dic_test})

In [250]:
species_dataset_lin.save_to_disk(os.path.join(data_path,"Species_linnaeus_dataset_ner"))

Saving the dataset (0/1 shards):   0%|          | 0/11935 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4078 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7142 [00:00<?, ? examples/s]

In [201]:
doc_seq,doc_labels,lines=readannotation_tsv(os.path.join(os.path.join(data_path,"NCBI-disease"),"train.txt"))


Identification	NN	O	O



In [209]:
print(doc_seq[0])

['Identification', 'of', 'APC2', ',', 'a', 'homologue', 'of', 'the', 'adenomatous', 'polyposis', 'coli', 'tumour', 'suppressor', '.']


In [204]:
lines[0:15]

['Identification\tNN\tO\tO\n',
 'of\tNN\tO\tO\n',
 'APC2\tNN\tO\tO\n',
 ',\tNN\tO\tO\n',
 'a\tNN\tO\tO\n',
 'homologue\tNN\tO\tO\n',
 'of\tNN\tO\tO\n',
 'the\tNN\tO\tO\n',
 'adenomatous\tNN\tO\tB-Disease\n',
 'polyposis\tNN\tO\tI-Disease\n',
 'coli\tNN\tO\tI-Disease\n',
 'tumour\tNN\tO\tI-Disease\n',
 'suppressor\tNN\tO\tO\n',
 '.\tNN\tO\tO\n',
 '\n']

In [207]:
doc_seq,doc_labels=readannotation_tsv(os.path.join(os.path.join(data_path,"NCBI-disease"),"train.txt"))
doc_seq_t,doc_labels_t=readannotation_tsv(os.path.join(os.path.join(data_path,"NCBI-disease"),"test.txt"))
doc_seq_v,doc_labels_v=readannotation_tsv(os.path.join(os.path.join(data_path,"NCBI-disease"),"dev.txt"))

Identification	NN	O	O

Clustering	NN	O	O

BRCA1	NN	O	O



In [191]:
len(doc_seq),len(doc_labels),len(doc_seq_v),len(doc_labels_v),len(doc_seq_t),len(doc_labels_t)

(5424, 5424, 923, 923, 940, 940)

In [223]:
entities_list,labels_list=getCleanEntities(doc_seq=doc_seq,doc_labels=doc_labels)
entities_list_test,labels_list_test=getCleanEntities(doc_seq=doc_seq_t,doc_labels=doc_labels_t)
entities_list_valid,labels_list_valid=getCleanEntities(doc_seq=doc_seq_v,doc_labels=doc_labels_v)

100%|██████████| 923/923 [00:00<00:00, 86750.83it/s]


In [224]:
doc_seq[0]

['Identification',
 'of',
 'APC2',
 ',',
 'a',
 'homologue',
 'of',
 'the',
 'adenomatous',
 'polyposis',
 'coli',
 'tumour',
 'suppressor',
 '.']

In [225]:
entities_list[0]

['Identification',
 'of',
 'APC2',
 ',',
 'a',
 'homologue',
 'of',
 'the',
 'adenomatous',
 'polyposis',
 'coli',
 'tumour',
 'suppressor',
 '.']

In [227]:
dic_train=datasets.Dataset.from_dict({"tokens":entities_list,"ner_tags":labels_list})
dic_test=datasets.Dataset.from_dict({"tokens":entities_list_test,"ner_tags":labels_list_test})
dic_val=datasets.Dataset.from_dict({"tokens":entities_list_valid,"ner_tags":labels_list_valid})
ncbiDisease_dataset=datasets.DatasetDict({"train":dic_train,"valid":dic_val,"test":dic_test})

In [228]:
ncbiDisease_dataset.save_to_disk(os.path.join(data_path,"Disease_NCBI-disease_dataset_ner"))

Saving the dataset (0/1 shards):   0%|          | 0/5424 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/923 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/940 [00:00<?, ? examples/s]

In [132]:
df = pd.read_csv(os.path.join(os.path.join(data_path,"s800"),"train.tsv"), delimiter='\t',header=None)

In [135]:
df.head(100)

,0,1
0,Methanoregula,B
1,formicica,I
2,sp,O
3,.,O
4,nov,O
...,...,...
95,;,O
96,cells,O
97,were,O
98,sometimes,O


In [134]:
df[1].value_counts()

1
O    141402
I      3310
B      2557
Name: count, dtype: int64

In [140]:
docs_seq_l=[]
doc_label_l=[]
word_seq=[]
label_seq=[]
for i in range(len(df)):
    if df[0][i]!="Abstract":
        word_seq.append(df[0][i])
        label_seq.append(df[1][i])
    else:
        doc_label_l.append(label_seq)
        docs_seq_l.append(word_seq)
        word_seq=[]
        label_seq=[]

In [141]:
len(docs_seq_l)

1

In [142]:
print([i for i in range(len(doc_label_l[2]))])
print(docs_seq_l[2])
print(doc_label_l[2])

IndexError: list index out of range

In [102]:
df[0].tolist().index('Abstract')

28

In [144]:
print(df[0].tolist()[0:500])

['Methanoregula', 'formicica', 'sp', '.', 'nov', '.', ',', 'a', 'methane', '-', 'producing', 'archaeon', 'isolated', 'from', 'methanogenic', 'sludge', '.', 'A', 'novel', 'methane', '-', 'producing', 'archaeon', ',', 'strain', 'SMSP', '(', 'T', ')', ',', 'was', 'isolated', 'from', 'an', 'anaerobic', ',', 'propionate', '-', 'degrading', 'enrichment', 'culture', 'that', 'was', 'originally', 'obtained', 'from', 'granular', 'sludge', 'in', 'a', 'mesophilic', 'upflow', 'anaerobic', 'sludge', 'blanket', '(', 'UASB', ')', 'reactor', 'used', 'to', 'treat', 'a', 'beer', 'brewery', 'effluent', '.', 'Cells', 'were', 'non', '-', 'motile', ',', 'blunt', '-', 'ended', ',', 'straight', 'rods', ',', '1', '.', '0', '-', '2', '.', '6', 'mum', 'long', 'by', '0', '.', '5', 'mum', 'wide', ';', 'cells', 'were', 'sometimes', 'up', 'to', '7', 'mum', 'long', '.', 'Asymmetrical', 'cell', 'division', 'was', 'observed', 'in', 'rod', '-', 'shaped', 'cells', '.', 'Coccoid', 'cells', '(', '0', '.', '5', '-', '1', '.'

In [21]:
df.head()

,,,-DOCSTART- (227508)
Naloxone,PROPN,O,I-Entity
reverses,VERB,O,O
the,DET,O,O
antihypertensive,ADJ,O,O
effect,NOUN,O,O


In [22]:
df.shape

(95079, 1)